#포켓몬 이름 맞추기
본 과제는 2점 만점이며, 정확도(Accuracy)를 평가 지표로 사용합니다.
Baseline보다 높은 정확도 달성: 2.0점
Baseline의 정확도와 동일(소수 5 번째 자리까지)하거나, 낮은 정확도 달성: 1.0점
미제출: 0점
(주의) 허용할 수 없을 정도로 낮은 정확도로 기록된 것도 미제출로 간주될 수 있습니다.

현재 Baseline은 Keras 라이브러리를 활용해 구성한 합성곱 신경망 모델을 활용해 도출됐습니다.
데이터 세트 분할 시, random_state 값은 42로 지정했습니다.
수업 시간에 다뤘던 합성곱 신경망 모델 구조 설계 및 하이퍼파라미터와 관련된 내용을 복습하고, 과제 5를 해결해 봅시다.

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# 1. 데이터 로드
data = np.load("train.npz")
x = data["x"]      # (292, H, W, 3) 이거나 (292, H, W)
y = data["y"]      # 문자열 라벨 (포켓몬 이름)

print("x shape before:", x.shape)

# 2. x 전처리 (채널 처리 + 정규화)
x = x.astype("float32") / 255.0

if x.ndim == 3:
    # (N, H, W) -> (N, H, W, 1)
    x = x[..., np.newaxis]
elif x.ndim == 4:
    # (N, H, W, C) 이미 채널 있음 -> 그대로 사용
    pass
else:
    raise ValueError(f"예상 밖의 x.ndim: {x.ndim}")

print("x shape after:", x.shape)  # 여기서 첫 번째 차원은 반드시 292여야 함

# 3. 레이블 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)          # 예: bulbasaur -> 0, charmander -> 1 ...
num_classes = len(le.classes_)
y_onehot = to_categorical(y_encoded, num_classes=num_classes)

# 4. train / validation 분할
x_train, x_val, y_train, y_val = train_test_split(
    x, y_onehot,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded   # stratify는 y_onehot 말고 y_encoded로!
)


x shape before: (292, 200, 200, 3)
x shape after: (292, 200, 200, 3)


In [7]:
# 5. CNN 모델 구성
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=x_train.shape[1:]),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[es],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print("Validation Accuracy:", val_acc)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.3233 - loss: 15.4547 - val_accuracy: 0.1864 - val_loss: 2.8151
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.7332 - loss: 5.8713 - val_accuracy: 0.2881 - val_loss: 3.9920
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8057 - loss: 4.1552 - val_accuracy: 0.2542 - val_loss: 6.7910
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8675 - loss: 2.0574 - val_accuracy: 0.2712 - val_loss: 8.9515
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8617 - loss: 2.2112 - val_accuracy: 0.1186 - val_loss: 11.9365
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.8782 - loss: 1.9275 - val_accuracy: 0.1525 - val_loss: 17.6989
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9079 - loss: 2.0307 - val_accuracy: 0.1186 - val_loss: 14.8547
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9074 - loss: 3.1340 - val_accuracy: 0.2373 - val_l

In [8]:
# 전체 데이터로 다시 학습 (선택)
model_final = model  # 그냥 이 모델 써도 되고, 원하면 x,y 전체로 다시 fit 해도 됨

# test 데이터 로드
test_data = np.load("test.npz")
x_test = test_data["x"].astype("float32") / 255.0

if x_test.ndim == 3:
    x_test = x_test[..., np.newaxis]
elif x_test.ndim == 4:
    pass
else:
    raise ValueError(f"예상 밖의 x_test.ndim: {x_test.ndim}")

# 예측
probs = model_final.predict(x_test)
pred_idx = probs.argmax(axis=1)
pred_labels = le.inverse_transform(pred_idx)  # 숫자를 다시 포켓몬 이름으로

# submission.csv에 쓰기
df = pd.read_csv("submission.csv")
df.dropna(axis=1, inplace=True)

df["result"] = pred_labels
df.to_csv("new_submission.csv", index=False)
print("✅ new_submission.csv 저장 완료")


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step   
✅ new_submission.csv 저장 완료


#2트

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 1) train.npz 로드
data = np.load("train.npz")
x = data["x"]      # (292, H, W) or (292, H, W, 3)
y = data["y"]      # 문자열 포켓몬 이름

print("x shape:", x.shape)
print("y shape:", y.shape)

# 2) 레이블 인코딩 (문자열 -> 숫자)
le = LabelEncoder()
y_idx = le.fit_transform(y)
num_classes = len(le.classes_)
print("클래스 개수:", num_classes)

# 3) 이미지 전처리
#   - grayscale이면 채널 추가
#   - 3채널 아니면 3채널로 맞추기
x = x.astype("float32")

if x.ndim == 3:
    # (N, H, W) -> (N, H, W, 1)
    x = x[..., np.newaxis]

if x.shape[-1] == 1:
    # (N, H, W, 1) -> (N, H, W, 3)
    x = np.repeat(x, 3, axis=-1)

print("x after channel fix:", x.shape)

# EfficientNet 입력 크기 (적당히 128 정도로)
IMG_SIZE = 128
x = tf.image.resize(x, (IMG_SIZE, IMG_SIZE)).numpy()

# 4) train / valid split (random_state=42 필수)
x_train, x_val, y_train, y_val = train_test_split(
    x, y_idx, test_size=0.2, random_state=42, stratify=y_idx
)

print("train:", x_train.shape, "valid:", x_val.shape)


x shape: (292, 200, 200, 3)
y shape: (292,)
클래스 개수: 8
x after channel fix: (292, 200, 200, 3)
train: (233, 128, 128, 3) valid: (59, 128, 128, 3)


In [10]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input

# 데이터 증강 레이어
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# 입력 정의
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x_in = data_augmentation(inputs)
x_in = preprocess_input(x_in)

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=x_in
)
base_model.trainable = False  # 1단계: 상단만 학습

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

es = EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True
)
rlr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    min_lr=1e-5
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[es, rlr],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print("1단계 (frozen base) Validation Accuracy:", val_acc)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 128, 128,  │          0 │ input_layer_1[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 128, 128,  │          0 │ sequential_1[0][… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 128, 128,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 128, 128,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 129, 129,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 64, 64,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 64, 64,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 64, 64,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 64, 64,    │          0 │ block1a_activati

 Total params: 4,059,819 (15.49 MB)

 Trainable params: 10,248 (40.03 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 705ms/step - accuracy: 0.2530 - loss: 2.1009 - val_accuracy: 0.4915 - val_loss: 1.6057 - learning_rate: 0.0010
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 467ms/step - accuracy: 0.5432 - loss: 1.5143 - val_accuracy: 0.7458 - val_loss: 1.2034 - learning_rate: 0.0010
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 493ms/step - accuracy: 0.7009 - loss: 1.1940 - val_accuracy: 0.8644 - val_loss: 0.9367 - learning_rate: 0.0010
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 542ms/step - accuracy: 0.8296 - loss: 0.8642 - val_accuracy: 0.9322 - val_loss: 0.7418 - learning_rate: 0.0010
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 419ms/step - accuracy: 0.8675 - loss: 0.7199 - val_accuracy: 0.9492 - val_loss: 0.6214 - learning_rate: 0.0010
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 522ms/step - accuracy: 0.9388 - loss: 0.5274 - val_accuracy: 0.9492 - val_loss: 0.5426 - learning_rate: 0.0010
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 412ms/step - accuracy: 0.9094 - loss: 0.5307

In [11]:
# EfficientNet 일부 레이어만 풀어서 미세조정
base_model.trainable = True

# 너무 많이 풀면 과적합이라 끝에서 몇 개 블록만 학습
fine_tune_at = len(base_model.layers) - 40  # 맨 끝 40개 레이어만 풀기

for i, layer in enumerate(base_model.layers):
    layer.trainable = (i >= fine_tune_at)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

es2 = EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True
)

history_ft = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    batch_size=16,
    callbacks=[es2],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print("파인튜닝 후 Validation Accuracy:", val_acc)


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 31s 839ms/step - accuracy: 0.6964 - loss: 1.0834 - val_accuracy: 0.9492 - val_loss: 0.5936
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 598ms/step - accuracy: 0.7545 - loss: 1.0669 - val_accuracy: 0.9492 - val_loss: 0.5784
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 590ms/step - accuracy: 0.6722 - loss: 1.0938 - val_accuracy: 0.9492 - val_loss: 0.5728
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 627ms/step - accuracy: 0.6869 - loss: 1.1075 - val_accuracy: 0.9322 - val_loss: 0.5707
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 663ms/step - accuracy: 0.7431 - loss: 0.9671 - val_accuracy: 0.9153 - val_loss: 0.5720
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 680ms/step - accuracy: 0.8083 - loss: 0.8960 - val_accuracy: 0.9153 - val_loss: 0.5745
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 535ms/step - accuracy: 0.7687 - loss: 0.9710 - val_accuracy: 0.9153 - val_loss: 0.5785
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 630ms/step - accuracy: 0.8173 - loss: 0.8822 - val_accuracy:

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder  # 이미 위에서 le 만들었으면 이 줄은 생략

# 이미 위에서: le, model, IMG_SIZE 정의되어 있고,
# train 전처리 때 썼던 x 채널/리사이즈 방식 그대로 재사용해야 함.

# 1) test.npz 로드 + 전처리
test_data = np.load("test.npz")
x_test = test_data["x"].astype("float32")

if x_test.ndim == 3:
    x_test = x_test[..., np.newaxis]
if x_test.shape[-1] == 1:
    x_test = np.repeat(x_test, 3, axis=-1)

x_test = tf.image.resize(x_test, (IMG_SIZE, IMG_SIZE)).numpy()

# 2) 예측
probs = model.predict(x_test)
pred_idx = probs.argmax(axis=1)
pred_labels = le.inverse_transform(pred_idx)  # 숫자 -> 포켓몬 이름

# 3) submission.csv에 result 채우기
df = pd.read_csv("submission.csv")
df.dropna(axis=1, inplace=True)

df["result"] = pred_labels
df.to_csv("new_submission.csv", index=False)

print("✅ new_submission.csv 저장 완료")


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
✅ new_submission.csv 저장 완료


#3트

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ============================================
# 1. 데이터 로드
# ============================================
print("1. 데이터 로드 중...")
try:
    train_data = np.load("train.npz")
    x_train_full = train_data['x']
    y_train_full = train_data['y']   # 문자 라벨 (예: 'Abra')

    test_data = np.load("test.npz")
    x_test = test_data['x']

    df_submission = pd.read_csv("submission.csv")

    if 'pokemon' in df_submission.columns:
        result_col = 'pokemon'
    else:
        result_col = 'result'

    print(f"데이터 로드 완료. 예측은 '{result_col}' 컬럼에 기록됩니다.")

except FileNotFoundError as e:
    print(f"파일 오류: {e}")
    exit()

print(f"총 클래스 개수: {len(np.unique(y_train_full))}")
print(f"학습 샘플: {x_train_full.shape[0]}, 테스트 샘플: {x_test.shape[0]}")
print(f"이미지 shape: {x_train_full.shape[1:]}")

# ============================================
# 2. 전처리
# ============================================

# A. 정규화
x_train_full = x_train_full.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# B. 채널 맞추기
image_shape = x_train_full.shape[1:]
if len(image_shape) == 2:  # (H,W)
    x_train_full = np.expand_dims(x_train_full, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    input_shape = x_train_full.shape[1:]
else:
    input_shape = image_shape

print("CNN input shape:", input_shape)

# 🟡 C. 가장 중요한 부분: 문자 라벨 → 숫자 라벨 변환
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train_full)

print("라벨 인코딩 예시:", list(zip(y_train_full[:5], y_encoded[:5])))
num_classes = len(label_encoder.classes_)

# 원-핫 인코딩
y_onehot = to_categorical(y_encoded, num_classes=num_classes)

# D. train/validation split
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full,
    y_onehot,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print("train/val 분할 완료:", x_train.shape, x_val.shape)

# ============================================
# 3. 데이터 증강
# ============================================

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_generator = datagen.flow(x_train, y_train, batch_size=32)

# ============================================
# 4. CNN 모델 구성
# ============================================

def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Dropout(0.25),

        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),

        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = create_cnn(input_shape, num_classes)

model.compile(
    optimizer=Adam(0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ============================================
# 5. 모델 학습
# ============================================

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // 32,
    epochs=150,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print(f"\n검증 정확도: {val_acc:.4f}")

# ============================================
# 6. 제출 파일 생성
# ============================================

# test 데이터 예측
y_pred_onehot = model.predict(x_test)
y_pred_idx = np.argmax(y_pred_onehot, axis=1)

# 숫자 → 원래 포켓몬 이름으로 변환
y_pred_labels = label_encoder.inverse_transform(y_pred_idx)

df_submission[result_col] = y_pred_labels

submission_file = "my_submission_xai_05.csv"
df_submission.to_csv(submission_file, index=False)

print(f"\n✅ 제출 파일 생성 완료: {submission_file}")


1. 데이터 로드 중...
데이터 로드 완료. 예측은 'pokemon' 컬럼에 기록됩니다.
총 클래스 개수: 8
학습 샘플: 292, 테스트 샘플: 73
이미지 shape: (200, 200, 3)
CNN input shape: (200, 200, 3)
라벨 인코딩 예시: [(np.str_('Abra'), np.int64(0)), (np.str_('Eevee'), np.int64(4)), (np.str_('Bulbasaur'), np.int64(2)), (np.str_('Arcanine'), np.int64(1)), (np.str_('Arcanine'), np.int64(1))]
train/val 분할 완료: (233, 200, 200, 3) (59, 200, 200, 3)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    34,669,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         4,104 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,766,408 (132.62 MB)

...

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.1167 - loss: 8.8902 - val_accuracy: 0.1356 - val_loss: 2.0786 - learning_rate: 0.0010
Epoch 2/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0938 - loss: 2.1143 - val_accuracy: 0.1356 - val_loss: 2.0789 - learning_rate: 0.0010
Epoch 3/150


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 362ms/step - accuracy: 0.1680 - loss: 2.0745 - val_accuracy: 0.1695 - val_loss: 2.0548 - learning_rate: 0.0010
Epoch 4/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2188 - loss: 2.0243 - val_accuracy: 0.1695 - val_loss: 2.0517 - learning_rate: 0.0010
Epoch 5/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - accuracy: 0.1489 - loss: 2.0475 - val_accuracy: 0.1864 - val_loss: 2.0206 - learning_rate: 0.0010
Epoch 6/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1250 - loss: 2.0265 - val_accuracy: 0.2881 - val_loss: 2.0172 - learning_rate: 0.0010
Epoch 7/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 280ms/step - accuracy: 0.1908 - loss: 1.9895 - val_accuracy: 0.2881 - val_loss: 1.9785 - learning_rate: 0.0010
Epoch 8/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.2812 - loss: 1.9414 - val_accuracy: 0.2712 - val_loss: 1.9665 - learning_rate: 0.0010
Epoch 9/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 291ms/step - accuracy: 0.3487 - loss: 1.7777 - val_accuracy: 0.3390 -